In [8]:
# !pip install pandas
# !pip install geodesic
# !pip install folium

In [9]:
import pandas as pd
from geopy.distance import geodesic

In [10]:
# Setup data
file_path_locations = "data/data_distance.csv"
file_path_boundaries = "data/data_boundaries.csv"
df_data_distance = pd.read_csv(file_path_locations)
df_data_boundaries = pd.read_csv(file_path_boundaries)
# df_data_distance.head()

In [11]:
def cal_distance_between_2_points(gdf, index1, index2):
    """
    cal_distance_between_2_points
    """
    coords_1 = (gdf.loc[index1,"lat"], gdf.loc[index1,"lng"])
    coords_2 = (gdf.loc[index2,"lat"],  gdf.loc[index2,"lng"])
    distance = geodesic(coords_1, coords_2).meters
    return distance

In [12]:
distances = []
for i in range(len(df_data_distance)):
    distance = 0
    for j in range(len(df_data_distance)):
        distance += cal_distance_between_2_points(df_data_distance,i,j)
    distances.append(distance)

df_data_distance['distances_total (m)'] = distances
df_data_distance['distances_avg (m)'] = df_data_distance['distances_total (m)']/(len(df_data_distance)-1)

df_data_distance

,lat,lng,distances_total (m),distances_avg (m)
0,20.920474,104.367553,21298.332417,10649.166208
1,21.041575,104.303014,33893.782713,16946.891356
2,20.917712,104.428089,25204.925957,12602.462979


### Visualize on Map


In [13]:
import folium
from folium import Circle

In [14]:
str_popup = "{}, {}"
m = folium.Map(location=[df_data_distance.loc[0,"lat"], df_data_distance.loc[0,"lng"],], zoom_start=11)

# marker_cluster = folium.MarkerCluster().add_to(m)


# boundaries map
for i in range(0,len(df_data_boundaries)):
    Circle(
        location=[df_data_boundaries.iloc[i]['lat'], df_data_boundaries.iloc[i]['lng']],
        radius=20,
        color='green',
        popup=str_popup.format(df_data_boundaries.iloc[i]['lat'], df_data_boundaries.iloc[i]['lng'])).add_to(m)

# location map
for id,row in df_data_distance.iterrows():
    folium.Marker(location=[row['lat'],row['lng']],
                  popup=str_popup.format(row['lat'],row['lng'])).add_to(m)
m